In [2]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

In [10]:
map = {}
class Data:
    # lay du lieu tu trong file
    def __init__(self, filename):
        lines = open(filename).read().split('\n')
        i = 0
        n = 1
        for line in lines:
            i = i + 1
            if i >= 7 and line != 'EOF' and line != '':
                type1 = [float(_) for _ in line.split()]
                map[n] = type1
                n = n + 1

In [37]:
class GA:
    def distance(self, city1, city2):
        if city1 in map and city2 in map:
            point1 = map[city1]
            point2 = map[city2]
            distance = np.power(((point1[1]-point2[1])**2 + (point1[2]-point2[2])**2), 0.5)
        return distance
    
    def routeDistance(self, route):
        distance = 0
        for i in range(0, len(route)):
            fromCity = route[i]
            toCity = None
            if i + 1 < len(route):
                toCity = route[i + 1]
            else:
                toCity = route[0]
            distance += self.distance(fromCity, toCity)
        return distance
    
    def routeFitness(self, route):
        fitness = 1 / float(self.routeDistance(route))
        return fitness
    
    def createRoute(self):
        route = random.sample(list(map), len(map))
        return route
    
    def initialPopulation(self, popSize):
        population = []
        for i in range(popSize):
            population.append(self.createRoute())
        return population
    
    def rankRoutes(self, population):
        fitnessResults = {}
        for i in range(0,len(population)):
            fitnessResults[i] = self.routeFitness(population[i])
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
    
    def selection(self):
        # chon lua theo fitness = 1/distance, lua chon ngau nhien boi vong quay roullette
        # luu ti le phan tram tuong ung voi moi diem o percentFitness
        fitnessValue = 0
        for i in populationDistance:
            fitness[i] = float(1/populationDistance[i])
            fitnessValue += float(1/populationDistance[i])
            percentFitness[i] = fitnessValue
        for m in fitness:
            percentFitness[m] = float(percentFitness[m]*100)/fitnessValue
        parent = {}
        i = 1
        while const >= i:
            rand = float(random.randrange(0, 10000))/100
            parent[i] = 1
            for n in percentFitness:
                if percentFitness[n] < rand and percentFitness[n-1] >= rand:
                    parent[i] = n - 1
            out = 0
            for m in parent:
                if parent[m] == parent[i] and m != i:
                    out = 1
            if out == 0:
                i = i + 1
            if out != 0:
                i = i
        return parent

In [51]:
print ("File data: ")
# filename = raw_input ("")
filename = "a280.tsp"
a = Data(filename) 
test = GA()
# route = [274, 217, 250, 58, 72, 47, 33, 141, 25, 26, 125, 14, 127, 18, 105, 188, 168, 118, 265, 205, 229, 224, 218, 15, 249, 238, 227, 221, 198, 214, 136, 280, 252, 209, 8, 150, 269, 137, 4, 5, 246, 142, 164, 74, 19, 89, 87, 17, 22, 91, 61, 131, 9, 20, 186, 104, 122, 76, 98, 60, 197, 267, 12, 148, 126, 161, 201, 256, 243, 237, 278, 255, 175, 174, 183, 102, 211, 203, 109, 272, 279, 167, 119, 40, 165, 231, 78, 154, 84, 176, 39, 116, 262, 264, 50, 16, 7, 99, 128, 277, 226, 155, 124, 185, 178, 85, 117, 216, 173, 143, 135, 79, 30, 24, 199, 273, 275, 94, 149, 53, 193, 3, 70, 115, 263, 276, 242, 65, 68, 110, 157, 152, 261, 239, 133, 140, 62, 71, 36, 171, 202, 247, 163, 121, 158, 213, 144, 151, 210, 206, 114, 194, 55, 82, 177, 153, 258, 101, 42, 268, 38, 106, 88, 54, 228, 92, 180, 138, 64, 112, 31, 132, 130, 215, 172, 160, 52, 113, 192, 1, 28, 222, 208, 59, 200, 96, 51, 83, 259, 166, 77, 75, 27, 43, 184, 108, 181, 123, 169, 182, 129, 271, 219, 232, 260, 251, 240, 48, 37, 179, 230, 241, 254, 212, 57, 146, 35, 170, 147, 93, 103, 45, 90, 111, 67, 63, 120, 13, 225, 195, 41, 233, 266, 139, 234, 56, 145, 236, 6, 235, 159, 162, 223, 66, 190, 245, 270, 248, 44, 46, 49, 80, 196, 97, 189, 134, 2, 95, 29, 69, 34, 21, 100, 187, 32, 73, 86, 220, 10, 253, 23, 107, 191, 11, 257, 204, 244, 81, 156, 207]
route = test.initialPopulation(3)
print(route)
for i in range(0,3):
    print(test.routeDistance(route[i]))
    print(test.routeFitness(route[i]))
currentRanked = test.rankRoutes(route)
print(currentRanked)

df = pd.DataFrame(np.array(currentRanked), columns=["Index", "Fitness"])
df['cum_sum'] = df.cumsum()
df['cum_perc'] = 100 * df.cum_sum / df.sum()
plt.plot(df)

File data: 
[[112, 124, 47, 269, 146, 38, 255, 8, 81, 92, 183, 186, 222, 57, 202, 90, 95, 167, 143, 94, 86, 14, 245, 77, 259, 263, 223, 10, 42, 199, 185, 178, 206, 265, 154, 28, 235, 278, 101, 204, 116, 176, 193, 276, 273, 16, 173, 27, 49, 120, 61, 249, 157, 261, 105, 241, 230, 22, 236, 181, 75, 108, 78, 187, 76, 64, 174, 115, 266, 88, 228, 21, 70, 32, 37, 196, 1, 237, 205, 79, 153, 240, 267, 135, 197, 217, 34, 254, 138, 118, 44, 98, 139, 218, 39, 155, 33, 63, 262, 182, 225, 195, 213, 110, 168, 175, 270, 58, 248, 83, 258, 41, 214, 221, 142, 59, 107, 233, 55, 163, 29, 17, 220, 149, 280, 72, 128, 190, 201, 198, 130, 121, 158, 192, 152, 2, 54, 133, 208, 7, 156, 99, 279, 234, 56, 91, 162, 25, 164, 67, 275, 26, 215, 141, 271, 69, 60, 50, 36, 231, 257, 140, 277, 184, 129, 102, 172, 15, 93, 171, 89, 30, 114, 166, 210, 11, 160, 136, 144, 256, 23, 212, 272, 216, 43, 5, 35, 68, 9, 238, 264, 85, 268, 40, 53, 132, 246, 134, 151, 104, 46, 20, 97, 189, 161, 51, 148, 111, 66, 150, 48, 96, 232, 242, 1

ValueError: Wrong number of items passed 2, placement implies 1